# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mulchen,-37.7167,-72.2333,46.06,90,99,2.04,CL,1682412191
1,1,weno,7.4515,151.8468,80.55,89,100,3.44,FM,1682412174
2,2,okha,53.5739,142.9478,31.64,81,40,5.64,RU,1682412192
3,3,yellowknife,62.4560,-114.3525,34.32,75,20,10.36,CA,1682412040
4,4,hawaiian paradise park,19.5933,-154.9731,71.11,90,0,0.00,US,1682412194


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    hover_cols=["City"],
    geo=True,
    tiles="OSM",
    size="Humidity",
    alpha=0.5,
    colorbar=True,
    width=800,
    height=600,
    cmap="Rainbow"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] > 70) & (city_data_df['Max Temp'] < 80) & (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,hawaiian paradise park,19.5933,-154.9731,71.11,90,0,0.00,US,1682412194
149,149,hilo,19.7297,-155.0900,79.29,89,0,5.01,US,1682412290
197,197,mermoz boabab,14.7065,-17.4758,70.36,94,0,9.22,SN,1682412322
214,214,callao,-12.0667,-77.1500,73.72,83,0,8.05,PE,1682412333
292,292,puerto baquerizo moreno,-0.9000,-89.6000,77.34,82,0,7.65,EC,1682412390
319,319,haiku-pauwela,20.9219,-156.3051,72.05,85,0,3.44,US,1682412241
355,355,saint croix,17.7397,-64.7388,73.96,83,0,4.61,VI,1682412430
407,407,el abiodh sidi cheikh,32.8930,0.5484,73.80,33,0,3.67,DZ,1682412463
416,416,acapulco de juarez,16.8634,-99.8901,76.82,83,0,0.00,MX,1682412469
473,473,ha'il,27.5219,41.6907,77.02,53,0,6.91,SA,1682412506


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", "")

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,mulchen,CL,-37.7167,-72.2333,90,
1,weno,FM,7.4515,151.8468,89,
2,okha,RU,53.5739,142.9478,81,
3,yellowknife,CA,62.4560,-114.3525,75,
4,hawaiian paradise park,US,19.5933,-154.9731,90,
...,...,...,...,...,...,...
544,nuku'alofa,TO,-21.1333,-175.2000,78,
545,ola,RU,59.5833,151.2833,81,
546,udomlya,RU,57.8794,34.9925,38,
547,teguise,ES,29.0605,-13.5640,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] =  f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mulchen - nearest hotel: Hostal LaTerraza
weno - nearest hotel: High Tide Hotel
okha - nearest hotel: Сфера
yellowknife - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
vilyuchinsk - nearest hotel: Победа
kavaratti - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
uturoa - nearest hotel: Hawaiki Nui hotel
blackmans bay - nearest hotel: Villa Howden
adamstown - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
carnarvon - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
isafjordur - nearest hotel: Hótel Horn
n'zeto - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
grytviken - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
ribeira grande - nearest hotel: Hospedaria JSF - 3 Estrelas
tiksi - nearest hotel: Арктика
tazovsky - nearest hotel: Тазовчан

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mulchen,CL,-37.7167,-72.2333,90,Hostal LaTerraza
1,weno,FM,7.4515,151.8468,89,High Tide Hotel
2,okha,RU,53.5739,142.9478,81,Сфера
3,yellowknife,CA,62.4560,-114.3525,75,No hotel found
4,hawaiian paradise park,US,19.5933,-154.9731,90,No hotel found
...,...,...,...,...,...,...
544,nuku'alofa,TO,-21.1333,-175.2000,78,City Hotel
545,ola,RU,59.5833,151.2833,81,No hotel found
546,udomlya,RU,57.8794,34.9925,38,Nomerok
547,teguise,ES,29.0605,-13.5640,68,Finca Las Laderas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [62]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 500,
    color = "City",
    scale = .5,
    hover_cols=['Country', 'Hotel Name']
)


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)